In [135]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [137]:
# Dataset parameters

train_dir = 'dataset/' # Path to your dataset directory
img_height = 224       # MobileNet default input size
img_width = 224        # MobileNet default input size
batch_size = 32


In [139]:
# Data augmentation and preprocessing
train_DG = ImageDataGenerator(
    rescale=1./255,             # Normalize pixel values to [0,1]
    validation_split=0.2,       # Split 20% for validation
)

In [141]:
# Load training data
train_images = train_DG.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',        # For two classes
    subset='training'           # For training data
)

Found 1600 images belonging to 2 classes.


In [143]:
# Load validation data
test_images = train_DG.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'         # For validation data
)

Found 400 images belonging to 2 classes.


In [145]:
base_model = MobileNet(
    weights='imagenet',         # Pre-trained on ImageNet
    include_top=False,          # Don't include the classifier
    input_shape=(img_height, img_width, 3)
)

In [147]:
# Create new model on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

In [149]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [151]:
# Model summary
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional) │ (None, 7, 7, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,360,193 (12.82 MB)

 Trainable params: 3,338,305 (12.73 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [153]:
# Train the model
history = model.fit(
    train_images,
    steps_per_epoch=train_images.samples // batch_size,
    validation_data=test_images,
    validation_steps=test_images.samples // batch_size,
    epochs=20
)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 630s 12s/step - accuracy: 0.8772 - loss: 0.3070 - val_accuracy: 0.8281 - val_loss: 0.7109
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 566s 11s/step - accuracy: 0.9693 - loss: 0.0739 - val_accuracy: 0.8125 - val_loss: 0.9005
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 531s 11s/step - accuracy: 0.9910 - loss: 0.0299 - val_accuracy: 0.7526 - val_loss: 1.4159
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 528s 11s/step - accuracy: 0.9887 - loss: 0.0405 - val_accuracy: 0.8021 - val_loss: 1.3788
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 532s 11s/step - accuracy: 0.9884 - loss: 0.0467 - val_accuracy: 0.8854 - val_loss: 0.5959
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 530s 11s/step - accuracy: 0.9833 - loss: 0.0438 - val_accuracy: 0.9089 - val_loss: 0.3788
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 532s 11s/step - accuracy: 0.9814 - loss: 0.0790 - val_accuracy: 0.8906 - val_loss: 0.4331
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 533s 11s/step - accuracy: 0.9986 - loss: 0.0040 - val_accuracy: 0.

In [179]:
# SavedModel format
model.save('mobilenet_health_leaf_model.h5')

In [181]:
test_loss, test_accuracy = model.evaluate(test_images)

13/13 ━━━━━━━━━━━━━━━━━━━━ 110s 8s/step - accuracy: 0.9655 - loss: 0.1712


In [176]:
print(f"\nReal Test Accuracy: {test_accuracy:.2%}") 
print(f"\nTest loss: {test_loss:.2%}" )


Real Test Accuracy: 97.25%

Test loss: 12.59%
